# Hackathon

Some utilities

## Import Utils

In [38]:
import keras
from keras.metrics import top_k_categorical_accuracy
import h5py as h5
import numpy as np
import pandas as pd
import tables

PATH_DATA = '/notebooks/data/half.h5'
OUT_PATH = '/notebooks/data/balanced.h5'

In [41]:
##DataSet Full
top_landcover = h5.File(PATH_DATA)['TOP_LANDCOVER']
top_landcover = top_landcover[0:].astype(int)
images = h5.File(PATH_DATA)['S2']

In [42]:
len(np.where(top_landcover == 17)[0][:])

86325

In [44]:
##Traitement des labels : 23 -> 14
unique, counts = np.unique(top_landcover, return_counts=True)
unique = pd.DataFrame(unique)
counts = pd.DataFrame(counts)
prop = pd.DataFrame(counts/18698240*100)

print(unique)
print(counts)

     0
0    1
1    2
2    3
3    4
4    5
5    6
6   10
7   11
8   12
9   13
10  14
11  15
12  17
13  18
14  19
15  20
          0
0   2583182
1    747083
2   1549478
3    216284
4   1998372
5        56
6     46350
7      9405
8   1745347
9        24
10    13603
11      152
12    86325
13     3329
14     9695
15   340435


In [51]:
classes = [1, 2, 3, 4, 5, 10, 11, 12, 14, 19, 20, 17]
indexes = np.zeros((len(classes), 100000))

for i in range(len(classes)):
        current_class = classes[i]  
        print(current_class)
        if current_class == 17:
            indexes[i][:86325] = np.where(top_landcover == current_class)[0][:]
        elif current_class == 10:
            indexes[i][:46350] = np.where(top_landcover == current_class)[0][:]
        elif current_class == 11:
            indexes[i][:9405] = np.where(top_landcover == current_class)[0][:]
        elif current_class == 14:
            indexes[i][:13603] = np.where(top_landcover == current_class)[0][:]
        elif current_class == 19:
            indexes[i][:9695] = np.where(top_landcover == current_class)[0][:]
        else:
            indexes[i][:] = np.where(top_landcover == current_class)[0][:100000]

indexes = indexes.flatten()
ind = np.where(indexes != 0 )
                
indexs = np.sort(indexes[ind])  
print(np.size(indexs))

1
2
3
4
5
10
11
12
14
19
20
17
865377


In [52]:
## DataSet Equilibré
balanced_top_lancover = top_landcover[indexs.astype(int)]

In [ ]:
balanced_images = images[indexs.astype(int), :, :, :]

In [ ]:
import os 

def build_h5_file(top_lancover, images, h5_output_path):
    if os.path.exists(h5_output_path):
        os.remove(h5_output_path)
    f = h5.File(h5_output_path, 'w')
    top_landcover_submit = f.create_dataset("TOP_LANDCOVER", (len(top_lancover), 1), maxshape=(None, 1))
    top_landcover_submit[:, 0] = top_lancover[:,0]
    s2_submit = f.create_dataset("S2", (len(images), 16,16,4), maxshape=(None, 16,16,4))
    s2_submit[:,:,:,:] = images
    f.close()
    
    return 1

build_h5_file(balanced_top_lancover, balanced_images, OUT_PATH)